In [1]:
import pandas as pd
import requests
import os
from dotenv import load_dotenv

/var/folders/_0/jby73m917sl0rmqvz60jmxzh0000gn/T/ipykernel_14370/1408106030.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
env = ".env"
load_dotenv(env)

True

In [3]:
API_KEY = os.getenv('CHATPDF_KEY')
API_URL_UPLOAD = "https://api.chatpdf.com/v1/sources/add-file"
API_URL_CHAT = "https://api.chatpdf.com/v1/chats/message"

In [4]:
def enviarPDF(arquivo):
    files = [
        ('file', ('file', open(arquivo, 'rb'), 'application/octet-stream'))
    ]
    headers = {
        'x-api-key': API_KEY
    }

    response = requests.post(
        API_URL_UPLOAD, headers=headers, files=files)

    if response.status_code == 200:
        SRC_ID = response.json()['sourceId']
        # print('ARQUIVO ENVIADO COM SUCESSO!')
        # print('SOURCE_ID:', SRC_ID)
        return SRC_ID
    else:
        print('Status:', response.status_code)
        print('Error:', response.text)

In [5]:
SRC_ID = enviarPDF('./pdf/alan.pdf')

In [8]:

def perguntar(pergunta1, src_id):
    headers = {
        'x-api-key': API_KEY,
        "Content-Type": "application/json",
    }

    data = {
        'sourceId': src_id,
        'messages': [
            {
                'role': "user",
                'content': pergunta1,
            }
        ]
    }

    response = requests.post(
        API_URL_CHAT, headers=headers, json=data)

    if response.status_code == 200:
        RESPOSTA = response.json()['content']
        # print('Result:', RESPOSTA)
        return RESPOSTA
    else:
        print('Status:', response.status_code)
        print('Error:', response.text)


In [10]:
pergunta1 = perguntar('Qual é o nome completo do Alan?',  SRC_ID)
pergunta2 = perguntar('Qual é o nome completo do Alan?', SRC_ID)

In [11]:
arquivos = os.listdir('./pdf/')
print(arquivos)
qtd_arquivos = len(os.listdir('./pdf/'))
print(qtd_arquivos)

['alan.pdf']
1


In [12]:
dados = pd.DataFrame()

for arquivo in arquivos:
    selecionado = './pdf/' + arquivo
    print(selecionado)
    src_id_enviado = enviarPDF(selecionado)

    resposta1 = perguntar('Qual é o nome completo do Alan?',  SRC_ID)
    resposta2 = perguntar('Qual é a profissão do Alan?', SRC_ID)
    nova_linha = pd.DataFrame({'Arquivo': [arquivo], 'SourceId': [src_id_enviado], 'Resposta1': [resposta1], 'Resposta2': [resposta2]})
    dados = pd.concat([dados, nova_linha], ignore_index=True)


./pdf/alan.pdf


In [13]:
dados

,Arquivo,SourceId,Resposta1,Resposta2
0,alan.pdf,src_YnEsj03Hrtxu8llS4s3Jq,O nome completo do Alan é Alan Da Silva Ramalho.,A profissão de Alan é desenvolvedor de softwar...
